In [2]:
import pandas as pd
import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Load datasets
car_dekho_df = pd.read_csv("/Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/raw/CAR DETAILS FROM CAR DEKHO.csv")
car_details_v4_df = pd.read_csv("/Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/raw/car details v4.csv")
car_prices_df = pd.read_csv("/Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/raw/car_prices.csv")
supply_chain_df = pd.read_csv("/Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/raw/Car_SupplyChainManagementDataSet.csv")
logistics_df = pd.read_excel("/Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/raw/Supply chain logisitcs problem.xlsx", sheet_name="OrderList")

# Load metadata JSON
with open("/Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/raw/archive (13)/Car damages dataset/meta.json", "r") as f:
    meta_data = json.load(f)

In [4]:
import pandas as pd
import json
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Convert metadata into a DataFrame
image_meta_df = pd.DataFrame(meta_data["classes"])
image_meta_df.rename(columns={"title": "Part_Class", "id": "Defect_ID"}, inplace=True)

# ------------------ STEP 1: Handle Missing Values ------------------

# Fill missing numerical values with median
for df in [car_dekho_df, car_details_v4_df, car_prices_df, supply_chain_df, logistics_df]:
    df.fillna(df.median(numeric_only=True), inplace=True)

# Fill missing categorical values with "Unknown"
for df in [car_dekho_df, car_details_v4_df, car_prices_df, supply_chain_df, logistics_df]:
    df.fillna("Unknown", inplace=True)

# ------------------ STEP 2: Feature Engineering ------------------

# Convert Year into Car Age
car_details_v4_df["Car_Age"] = 2025 - car_details_v4_df["Year"]

# Normalize numerical values (Weight, Price, Odometer)
scaler = MinMaxScaler()
numerical_cols = ["Price", "Kilometer", "CarPrice", "Weight"]
for col in numerical_cols:
    if col in car_details_v4_df.columns:
        car_details_v4_df[col] = scaler.fit_transform(car_details_v4_df[[col]])

# Encode categorical columns
encoder = LabelEncoder()
categorical_cols = ["Make", "Model", "Fuel Type", "Transmission", "Carrier", "Service Level"]
for col in categorical_cols:
    if col in car_details_v4_df.columns:
        car_details_v4_df[col] = encoder.fit_transform(car_details_v4_df[col])

# ------------------ STEP 3: Fix Column Compatibility for Merging ------------------

# Convert 'OrderID' and 'Order ID' to string to ensure compatibility
supply_chain_df['OrderID'] = supply_chain_df['OrderID'].astype(str)
logistics_df['Order ID'] = logistics_df['Order ID'].astype(str)

# Use CarModel instead of ProductID for merging with car details
supply_chain_df['CarModel'] = supply_chain_df['CarModel'].astype(str)
car_details_v4_df['Model'] = car_details_v4_df['Model'].astype(str)

# Convert ProductID to string for merging with metadata
supply_chain_df['ProductID'] = supply_chain_df['ProductID'].astype(str)
image_meta_df['Defect_ID'] = image_meta_df['Defect_ID'].astype(str)

# ------------------ STEP 4: Merge Datasets ------------------

# Merge supply chain data with logistics using OrderID
merged_df = pd.merge(supply_chain_df, logistics_df, left_on="OrderID", right_on="Order ID", how="left")

# Merge with car details using CarModel instead of ProductID
merged_df = pd.merge(merged_df, car_details_v4_df, left_on="CarModel", right_on="Model", how="left")

# Merge with image metadata using ProductID (assuming Defect_ID corresponds to ProductID)
final_df = pd.merge(merged_df, image_meta_df, left_on="ProductID", right_on="Defect_ID", how="left")

# ------------------ STEP 5: Save Final Preprocessed Dataset ------------------

# Drop unnecessary columns
final_df.drop(columns=["Order ID", "Model", "Defect_ID"], inplace=True)

# Save to CSV
final_df.to_csv("/Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/processed/final_preprocessed_data1.csv", index=False)

print("✅ Data Preprocessing Completed! Merged dataset saved.")


✅ Data Preprocessing Completed! Merged dataset saved.


In [5]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Load the preprocessed dataset
preprocessed_data_path = '/Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/processed/final_preprocessed_data1.csv'
preprocessed_df = pd.read_csv(preprocessed_data_path)

# Load image metadata if needed
# Assuming 'meta_data' is a dictionary containing the metadata information
image_meta_df = pd.DataFrame(meta_data["classes"])
image_meta_df.rename(columns={"title": "Part_Class", "id": "Defect_ID"}, inplace=True)

# ------------------ STEP 1: Re-Merge Missing Data ------------------

# Convert necessary columns to string for merging
preprocessed_df['ProductID'] = preprocessed_df['ProductID'].astype(str)
image_meta_df['Defect_ID'] = image_meta_df['Defect_ID'].astype(str)

# Re-merging image metadata using ProductID
merged_df = pd.merge(preprocessed_df, image_meta_df, left_on="ProductID", right_on="Defect_ID", how="left")

# ------------------ STEP 2: Drop Columns with 100% Missing Values ------------------

# Drop columns where all values are NaN
cleaned_df = merged_df.dropna(axis=1, how='all')

# ------------------ STEP 3: Handle Missing Values ------------------

# Filling missing numerical values with median
numerical_columns = cleaned_df.select_dtypes(include=['float64', 'int64']).columns
imputer = SimpleImputer(strategy='median')
cleaned_df[numerical_columns] = imputer.fit_transform(cleaned_df[numerical_columns])

# Filling missing categorical values with "Unknown"
categorical_columns = cleaned_df.select_dtypes(include=['object']).columns
imputer_cat = SimpleImputer(strategy='constant', fill_value='Unknown')
cleaned_df[categorical_columns] = imputer_cat.fit_transform(cleaned_df[categorical_columns])

# ------------------ STEP 4: Convert Date Fields ------------------

# Convert date fields to datetime format
date_columns = ['OrderDate', 'ShipDate']
for col in date_columns:
    if col in cleaned_df.columns:
        cleaned_df[col] = pd.to_datetime(cleaned_df[col], errors='coerce')

# ------------------ STEP 5: Save the Cleaned Dataset ------------------

# Save the cleaned and merged dataset
cleaned_data_path = '/Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/processed/cleaned_final_preprocessed_data.csv'
cleaned_df.to_csv(cleaned_data_path, index=False)

print(f"✅ Data Cleaning and Re-merging Completed! Cleaned dataset saved at {cleaned_data_path}")


✅ Data Cleaning and Re-merging Completed! Cleaned dataset saved at /Users/adityasrivatsav/Documents/GitHub/-SmartStock-AI-powered-inventory-optimization-and-demand-forecasting/data/processed/cleaned_final_preprocessed_data.csv


/var/folders/r5/_bzgrbtj6tg33lk21qcyg0l80000gn/T/ipykernel_21346/3006025990.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df[numerical_columns] = imputer.fit_transform(cleaned_df[numerical_columns])
/var/folders/r5/_bzgrbtj6tg33lk21qcyg0l80000gn/T/ipykernel_21346/3006025990.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df[categorical_columns] = imputer_cat.fit_transform(cleaned_df[categorical_columns])
/var/folders/r5/_bzgrbtj6tg33lk21qcyg0l80000gn/T/ipykernel_21346/3006025990.